### Short description of model preprocessing 

- Divide on parts
    - Why we add 195 after all
    - delay between EEG and fMRI. Which one? 
- Make sFFT on eeg
    - 2-19 WHy only thes freq. 
    - nperseg is 400 &&
- Make interpolation of fMRI
- dataset i

### Cons ( Minus )
- linear combination of amplitude features. Why don't use phase information
- No linear is may be better
- Different models for different zones (ROI of fMRI )
Work independently. No time consistency
- FMRI depend not only EEG but of course on previous inforamtion 
- This was my error when i work on decoding problem. 
- ALSO. different areas of MRI can influence on each other. 



### Approaches
Some alterantive of attention is quite interersting for this task. Because we can understand how different region related and which activivty in which time affect on signals

Data augmentation
- Add noise 
- Add noise in freq
    - Channel wise
    - Time wise
    - there is paper with it .  inpainting.
- Make channel flip !!?
- 

Regularization
- L1 it allows get sparse matrix 
- Of course dropout

Extra note
- Solve no regression but classification problem 

**NOTE**. Based on EEG activity we can predict not precisely activity od small region but distriution of energy. For example. Classify which region is active.
Multiclass multi object. classification. Interresting yeh.
We can begin with simple softmax on the last layer. 


In [1]:
import sys
import sklearn
import mne

from nilearn import image
import numpy as np
import matplotlib.pyplot as plt


#my codes 
from fMRI_Regions import get_masked_fmri
import original_input
import make_input

0.01

In [2]:
eeg_path = '../data/eeg_fmri_data_trio_four_subjects/eeg_on/eeg_on_1.edf'
fmri_path = '../data/eeg_fmri_data_trio_four_subjects/fmri_on/fmri_on_1.nii'

args = {'start_time': 0, 
        'end_time': 10, 
        'delay': 1, 
        'fmri_end': 10,
        'eeg_path': eeg_path,
        'fmri_path': fmri_path}

## Read RAW files files




Ощущение колхозности не отсутпает от меняю 

In [18]:
from preprocessing import eeg_transform, fmri_transform

In [24]:
segment_length = 16384
def create_dataset(start_time, end_time, delay, fmri_end, eeg_path, fmri_path):
    vector_exclude = ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6', 'Status']
    raw = mne.io.read_raw_edf(eeg_path, exclude=vector_exclude)
    eeg = raw.get_data()
    eeg = mne.filter.filter_data(eeg, sfreq=1000, l_freq=5, h_freq=100)
    print('step1', eeg.shape)
    
    eeg_flip = np.fliplr(eeg)
    fmri_im = image.smooth_img(fmri_path, fwhm=6)
    fmri = get_masked_fmri(fmri_im, "sub")
    print('step2', fmri.shape)
    
    start = start_time
    end = start_time + segment_length
    x_list = []
    y_list = []
    x_fl_list = []
    while end < eeg.shape[1] and end <= fmri_end and end < end_time:
        signal = eeg[..., start:end]
#         print(signal.shape)
        signal_flip = eeg_flip[..., start:end]
        x = eeg_transform(signal)
        y = fmri_transform(end, fmri, delay, fmri_end)
        x1 = eeg_transform(signal_flip)
        x_list.append(x)
        y_list.append(y)
        x_fl_list.append(x1)
        # why 195. What is it???
        start += 195
        end += 195
    x_list = np.array(x_list)
    x_fl_list = np.array(x_fl_list)
    y_list = np.array(y_list)
    return x_list, y_list, x_fl_list


In [25]:
args = {'start_time': 0, 
        'end_time': 100000, 
        'delay': 0, 
        'fmri_end': 100000,
        'eeg_path': eeg_path,
        'fmri_path': fmri_path}
x_list, y_list, x_fl_list = create_dataset(**args)
print(x_list.shape, y_list.shape, x_fl_list.shape)

Extracting EDF parameters from D:\Work\HSE_project\EEG_fMRI\data\eeg_fmri_data_trio_four_subjects\eeg_on\eeg_on_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Setting up band-pass filter from 5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1651 samples (1.651 sec)

step1 (30, 305000)
step2 (150, 21)
(429, 30, 17, 60) (429, 21) (429, 30, 17, 60)


In [30]:
eeg[..., 10:20].shape

(30, 10)

In [3]:
creater = make_input.Dataset()
x_list, y_list, x_fl_list = creater.create_dataset(**args)


Extracting EDF parameters from D:\Work\HSE_project\EEG_fMRI\data\eeg_fmri_data_trio_four_subjects\eeg_on\eeg_on_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Setting up band-pass filter from 5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1651 samples (1.651 sec)



In [4]:
print(x_list, y_list.shape, x_fl_list.shape)

[] (0,) (0,)
